### Packages

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import numpy.random as nr
import itertools as it
import operator as op
import random as rd

### Input

In [2]:
N = int(input('Input number of airports'))

Input number of airports 100


In [3]:
PopulationSize = 2
EliteSize = 1
MutationRate = 0.0005 
generation = 10000

### Data

In [4]:
toc = dt.datetime.now() #program start time

nr.seed(20220606)
rd.seed(20220606)

#flight costs matrix
C = 708 + (51124 - 708)*nr.rand(N**2).reshape(N,N)
C -= np.diag(np.diag(C)) 
        
interval = {}
interval["Acheck"] = nr.randint(56, 71)
interval["Bcheck"] = nr.randint(180, 241)
interval["Ccheck"] = nr.randint(540, 721)
interval["Dcheck"] = nr.randint(2160, 3601)
        
durations = {}
durations["Acheck"] = [1]*N
durations["Bcheck"] = nr.randint(1, 4, size = N)
durations["Ccheck"] = nr.randint(7, 15, size = N)
durations["Dcheck"] = nr.randint(21, 43, size = N)
        
costs = {}
costs["Acheck"] = (310 + (820 - 310)*nr.rand(N))
costs["Bcheck"] = (4960 + (7380 - 4960)*nr.rand(N))
costs["Ccheck"] = (186000 + (246000 - 186000)*nr.rand(N))
costs["Dcheck"] = (930000 + (2050000 - 930000)*nr.rand(N))

MFPD = 4 #maximum flights per day
home = nr.randint(N) #initial airport
I = set(np.arange(N)) #airport set
MFCPD = C.max() #maximum flight costs per day
MFC = (C + np.diag([np.inf]*N)).min() #minimum flight costs
MDP = (MFCPD + MFPD*MFC)/2 #median daily profit

### Aircraft Simulation

In [5]:
def maintenance_check(day, maintenance, airport):
    for check in maintenance["amount"].keys():
        if day >= (maintenance["amount"][check]+1)*interval[check] :
            
            maintenance["amount"][check] += 1
            day += durations[check][airport]
            
            maintenance["cost"][check] += costs[check][airport]
            maintenance["loss"][check] += durations[check][airport]*MDP
            
    return day, maintenance


def total_cost(route):
    day = 0 # aircraft age
        
    maintenance = {}
    maintenance["loss"]     = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
    maintenance["amount"]   = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
    maintenance["cost"]     = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
        
    FC = 0 #flight cost
    
    FPD = 0 #flights per day
    FCPD = 0 #flight costs per day
        
    for i in range(1,len(route)): 
        FCPD += C[route[i-1]][route[i]]
        FPD += 1
        
        if (FPD > MFPD) or (FCPD > MFCPD):
            FC += (FCPD - C[route[i-1]][route[i]])
            day += 1
            
            FPD = 1
            FCPD = C[route[i-1]][route[i]]
            day, maintenance = maintenance_check(day, maintenance, route[i-1])
            
    FC += FCPD
    if route[-1] == home:
        day += 1
        day, maintenance = maintenance_check(day, maintenance, route[-1])
    
    objective = FC + sum(list(maintenance["cost"].values()))
    objective += sum(list(maintenance["loss"].values()))

    return day, FC, maintenance, objective

### Genetic Algorithm

In [6]:
def ranking(population):
    population_ranking = {}
    for i in range(len(population)):
        
        route = population[i]
        day, FC, maintenance, objective = total_cost(route)
        population_ranking[i] = 1/objective
        
    return sorted(population_ranking.items(), key = op.itemgetter(1), reverse = True)
    
    
def selection(population_ranking):
    arr_selection = []
    df = pd.DataFrame(np.array(population_ranking), columns=["indeks","fitness"])
    df['cum_sum'] = df.fitness.cumsum()/df.fitness.sum()
 
    for i in range(EliteSize):
        arr_selection.append(population_ranking[i][0])
    
    for i in range(len(population_ranking) - EliteSize):
        pick = rd.random()
        for i in range(len(population_ranking)):
            if pick <= df.iat[i,2]:
                arr_selection.append(population_ranking[i][0])
                break

    selection_results = []
    for i in range(len(arr_selection)):
        selection_results.append(population[arr_selection[i]])
    
    return selection_results
    
    
def crossover(parent1, parent2):
    child = []
    child1 = []
    child2 = []
    
    gene1 = int(rd.random() * len(parent1))
    gene2 = int(rd.random() * len(parent1))
    
    start = min(gene1, gene2)
    end = max(gene1, gene2)

    for i in range(start, end):
        child1.append(parent1[i])
        
    child2 = [i for i in parent2 if i not in child1]

    child = child1 + child2
    return child
    
    
    
def mutation(individual, MutationRate):
    for i in range(len(individual)):
        if(rd.random() < MutationRate):
            j = int(rd.random() * len(individual))
            
            city1 = individual[i]
            city2 = individual[j]
            
            individual[i] = city2
            individual[j] = city1
            
    return individual

### Main Program

In [7]:
#population initiation
population = []
for i in range(PopulationSize):
    
    route = [home] + rd.sample(list(I-set([home])), N-1) + [home]
    while route in population:
        route = [home] + rd.sample(list(I-set([home])), N-1) + [home]
        
    population.append(route)  

first_generation = int(1/ranking(population)[0][1])

for i in range(generation):
    population_ranking = ranking(population)
    selection_results = selection(population_ranking)
    
    children = []
    length = len(selection_results) - EliteSize
    pool = rd.sample(selection_results, len(selection_results))

    for i in range(EliteSize):
        children.append(selection_results[i])
    
    for i in range(length):
        child = [home]
        child += crossover(pool[i][1:-1], pool[len(selection_results)-i-1][1:-1])
        child += [home]
        mutation_result = [home] + mutation(child[1:-1], MutationRate) + [home]
        children.append(mutation_result)
    
    population = children


champion = ranking(population)[0]

indeks = champion[0]
fitness = champion[1]

route = population[indeks]
day, FC, maintenance, objective = total_cost(route)

final_generation = int(1/fitness)
generation_rate = first_generation/final_generation*100

In [8]:
def output():
    print("objective first generation: $" + str(first_generation))
    print("objective generation akhir : $" + str(final_generation))
    print('persentase generation : '+str(int(generation_rate))+'%')

    print()
    print('Total cost of flights  $%d' 
        %(FC))
    print('Total maintenance costs $%d' 
        %(sum(list(maintenance["cost"].values()))))
    print('Total maintenance loss $%d' 
        %(sum(list(maintenance["loss"].values()))))

    print('Total cost $%d' 
        %(FC + sum(list(maintenance["cost"].values()))))
    print()
    print("\033[1m" +"Details :" +"\033[0m")
    print(f'flight duration {int(day)} day')
    print()
    print('Number of The A check maintenance = %d' 
        %(maintenance["amount"]['Acheck']))
    print('Number of The B check maintenance = %d' 
        %(maintenance["amount"]['Bcheck']))
    print('Number of The C check maintenance = %d' 
        %(maintenance["amount"]['Ccheck']))
    print('Number of The D check maintenance = %d' 
        %(maintenance["amount"]['Dcheck']))
    print()
    print('Costs of The A check maintenance = $%d' 
        %(maintenance["cost"]['Acheck']))
    print('Costs of The B check maintenance = $%d' 
        %(maintenance["cost"]['Bcheck']))
    print('Costs of The C check maintenance = $%d' 
        %(maintenance["cost"]['Ccheck']))
    print('Costs of The D check maintenance = $%d' 
        %(maintenance["cost"]['Dcheck']))
    print()
    print('Loss of The A check maintenance = $%d' 
        %(maintenance["loss"]['Acheck']))
    print('Loss of The B check maintenance = $%d' 
        %(maintenance["loss"]['Bcheck']))
    print('Loss of The C check maintenance = $%d' 
        %(maintenance["loss"]['Ccheck']))
    print('Loss of The D check maintenance = $%d' 
        %(maintenance["loss"]['Dcheck']))
    print()

    print("route : ")
    for airport in route[:-1]:
        print(f'{airport} -> ', end ='')
       
    print(home)
    print()
    print("Execution time : ", dt.datetime.now()-toc)

def detail():
    print('Detail: ')
    FC = 0
    FC_per_day = 0
    flight_per_day = 0 
    day = 0
    for i in range(1,len(route)):
        FC_per_day += C[route[i-1]][route[i]]
        flight_per_day += 1
    
        if (flight_per_day > MFPD) or (FC_per_day > MFCPD):
            FC += (FC_per_day - C[route[i-1]][route[i]])
            day += 1
        
            print('day: '+str(day)+', airport visited: '+str(route[i-flight_per_day :i])+', flight costs: '+str(FC_per_day - C[route[i-1]][route[i]]))
            FC_per_day = C[route[i-1]][route[i]]
            flight_per_day = 1
        
    FC += FC_per_day
    day += 1
        
    print('day: '+str(day)+', airport visited: '+str(route[-flight_per_day :])+', flight costs: '+str(FC_per_day))
    print()
    print("Total cost of flights :",int(FC))

### Output

In [9]:
output()

objective first generation: $2288915
objective generation akhir : $558210
persentase generation : 410%

Total cost of flights  $558210
Total maintenance costs $0
Total maintenance loss $0
Total cost $558210

Details :
flight duration 25 day

Number of The A check maintenance = 0
Number of The B check maintenance = 0
Number of The C check maintenance = 0
Number of The D check maintenance = 0

Costs of The A check maintenance = $0
Costs of The B check maintenance = $0
Costs of The C check maintenance = $0
Costs of The D check maintenance = $0

Loss of The A check maintenance = $0
Loss of The B check maintenance = $0
Loss of The C check maintenance = $0
Loss of The D check maintenance = $0

route : 
80 -> 88 -> 11 -> 64 -> 71 -> 9 -> 90 -> 24 -> 63 -> 4 -> 10 -> 40 -> 20 -> 54 -> 96 -> 33 -> 3 -> 46 -> 53 -> 50 -> 56 -> 75 -> 57 -> 81 -> 32 -> 68 -> 47 -> 36 -> 67 -> 18 -> 93 -> 86 -> 45 -> 1 -> 12 -> 94 -> 52 -> 99 -> 35 -> 58 -> 70 -> 21 -> 60 -> 82 -> 37 -> 29 -> 87 -> 14 -> 85 -> 43 -

### Detail

In [10]:
detail()

Detail: 
day: 1, airport visited: [80, 88, 11, 64, 71], flight costs: 16145.193444592243
day: 2, airport visited: [71, 9, 90, 24, 63], flight costs: 18820.25173343798
day: 3, airport visited: [63, 4, 10, 40, 20], flight costs: 16269.633291713622
day: 4, airport visited: [20, 54, 96, 33, 3], flight costs: 18538.41572988929
day: 5, airport visited: [3, 46, 53, 50, 56], flight costs: 9786.737470695673
day: 6, airport visited: [56, 75, 57, 81, 32], flight costs: 14145.815655495695
day: 7, airport visited: [32, 68, 47, 36, 67], flight costs: 26141.234676516582
day: 8, airport visited: [67, 18, 93, 86, 45], flight costs: 45741.58202853256
day: 9, airport visited: [45, 1, 12, 94, 52], flight costs: 29338.521710874236
day: 10, airport visited: [52, 99, 35, 58, 70], flight costs: 37016.13731355048
day: 11, airport visited: [70, 21, 60, 82, 37], flight costs: 6911.449350192441
day: 12, airport visited: [37, 29, 87, 14, 85], flight costs: 24523.30676903961
day: 13, airport visited: [85, 43, 7, 17